# Jupyterlab version

# Extract and process data into nodes and edges

In [1]:
import pickle
import json
import gzip
from pprint import pprint
import os.path

# Import data

Assumes that the data files are in a subfolder called 'Data'

In [6]:
# Written by Alex
# unzips the file and goes through it
# 1 line is 1 post

data_path = '../Data/'  #change dir to your project folder
filename = os.path.join(data_path, 'processed.1850k.jsonl.gz')

def iterate_posts():
    with gzip.open(filename, 'rb') as fd:
      for line in fd:
          yield json.loads(line)
            
gen = iterate_posts()
#next(gen)

In [7]:
# Collect a set of all usernames found in posts
# Using posts.pickle.gz allows 1 line = 1 name
# This file breaks replies off from the parent post
# so that we don't have to worry about parsing multiple
# names in one line.
# Once we haved a list of each username from each post,
# remove duplicates to get a full list of unique usernames
from collections import Counter

filename2 = os.path.join(data_path, 'posts.pickle.gz')

single_posts = pickle.load(gzip.open(filename2, 'rb'))
nodes = []
for post in single_posts.values():
    nodes.append(post['author'].replace(" ", "_")) # Need to remove spaces in a few names
    
user_post_count = Counter(nodes)


In [ ]:
top_10k = user_post_count.most_common(10001)
top_10k = top_10k[1:10001]
len(top_10k)

In [ ]:
nodes = [item[0] for item in top_10k]

In [ ]:
#top_10k

In [ ]:
nodes[:100]

In [ ]:
def get_list_of_users(post):
    """ 
    Extract all usernames from each post

    Note:
        There can be None, one, or more usernames in a post.
     
     Args:
         post (dict): Yielded by iterate_posts()

    Attributes:
        users (dict): Used to collect usernames 
        
    Returns:
        users: Tuple of usernames from each post. Can contain any number of usernames.
    """    

    users = []
    try:
        for post_item in post['posts']:                
            users.append(post_item['author_username'].replace(" ", "_"))
    except KeyError:
        if len(users)>1:
            return tuple(users) #list of usernames from ONE(1) post
        else:
            return None

In [ ]:
# Collect a list of tuples of of length 2 or more usernames from posts
# Filters out lists of None and lists with only 1 user. This is why 'minimum_posts' of 2 is unneeded.
# The exception is triggered by reaching the end of the JSONL file.
def get_edges():
    user_tuple_list=[]
    edges_temp=[]

    try:
        while True:
            user_tuple_list.append(get_list_of_users(next(gen))) 
    except:  
        user_tuple_list = list(set(user_tuple_list))
        for user_list in user_tuple_list:
#            if user_list != None and len(user_list)>1:
            if user_list != None:
                edges_temp.append(user_list)
            else:
                continue
        print("Complete")
        return edges_temp

In [ ]:
def split_edges(all_edges):
    """  
    Split tuples into tuples of length 2 and remove duplicates.

    Args:
        all_edges (list of tuples): These tuples vary in length.

    Attributes: 
        user_list (list): Keeps tuples generated/passed by for loop.
        
    Returns:
        user_list (list of tuples): Each tuple is of length 2 and duplicates are removed.
        
    """

    user_list = []
    
    for user_tuple in all_edges:
        if len(user_tuple)==2:
            user_list.append(user_tuple)
        else:
            for username in user_tuple:
                if username!=user_tuple[0]:
                    user_list.append((user_tuple[0],username))
                else:
                    continue
            continue
    return list(set(user_list))

In [ ]:
def remove_low_posters(edges):
    temp_list = []
    for username_tuple in edges:
        name_list = []
        for username in username_tuple:
            if username in nodes:
                name_list.append(username)
            else:
                pass
        if len(name_list)>1:
            temp_list.append(tuple(name_list))
        else:
            pass
    return temp_list

In [ ]:
all_edges = get_edges()
print(f'all_edges is a {type(all_edges)} of {type(all_edges[0])} containing {type(all_edges[0][0])}')
len(all_edges)

In [ ]:
# Make all elements of edges to be tuples of length 2 with duplicates removed.
edges = split_edges(all_edges)
len(edges)

In [ ]:
high_posters = remove_low_posters(edges)
len(high_posters)

In [ ]:
len(high_posters)

In [ ]:
high_posters[:10]

In [ ]:
# save the nodes and edges to a file
# TODO: Write code to read in the files to the appropriate variables so that we don't have to run all of the above code again
graph_path = 'Graph/'

open(os.path.join(graph_path,'edges.txt'), 'w').write('\n'.join('%s %s' % x for x in high_posters))
#open(os.path.join(graph_path,'edges with low posters removed.txt'), 'w').write('\n'.join('%s %s' % x for x in high_posters))
open(os.path.join(graph_path,'nodes.txt'), 'w').write('\n'.join('%s' % x for x in nodes))